In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as kr

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [51]:
#read training data into DataFrame
df = pd.read_csv('processed_training_data_preprocessed.csv',encoding = "ISO-8859-1" )

In [52]:
df.head()

,polarity of tweet,tweet_text,clean_tweet
0,0,is upset that he can't update his Facebook by ...,is upset that he cannot update his facebook by...
1,0,@Kenichan I dived many times for the ball. Man...,dived many times for the ball managed to save ...
2,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all....",no it has it is not behaving at all am mad why...
4,0,@Kwesidei not the whole crew,not the whole crew


In [47]:
data = df[['clean_tweet','polarity of tweet']]
data.head()

,clean_tweet,polarity of tweet
0,is upset that he cannot update his facebook by...,0
1,dived many times for the ball managed to save ...,0
2,my whole body feels itchy and like its on fire,0
3,no it has it is not behaving at all am mad why...,0
4,not the whole crew,0


In [65]:
#data = df[['clean_tweet','polarity of tweet']]
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['clean_tweet'].values)
X = tokenizer.texts_to_sequences(data['clean_tweet'].values)
X = pad_sequences(X)


In [67]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 50, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
Y = pd.get_dummies(data['polarity of tweet']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1072332, 50) (1072332, 3)
(528164, 50) (528164, 3)


In [69]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

W0901 03:42:12.540051 139867592820480 deprecation.py:323] From /root/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/7
 - 1772s - loss: 0.4487 - acc: 0.7888
Epoch 2/7
 - 1757s - loss: 0.4263 - acc: 0.8025
Epoch 3/7
 - 1826s - loss: 0.4194 - acc: 0.8064
Epoch 4/7
 - 1780s - loss: 0.4159 - acc: 0.8086
Epoch 5/7
 - 1772s - loss: 0.4144 - acc: 0.8096
Epoch 6/7
 - 1787s - loss: 0.4131 - acc: 0.8100
Epoch 7/7
 - 1820s - loss: 0.4124 - acc: 0.8109


In [70]:
from keras.models import load_model

In [72]:
model.save('tweets_sentiment_classification.h5')

In [75]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("accuracy: %.2f" % (acc))

score: 0.41
accuracy: 0.81
